In [ ]:
import os
import pandas as pd
import torch 
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
from torchvision.models import resnet18, ResNet18_Weights

In [16]:
#Defining data location 
train_folder = r"C:\Users\Lenovo\Desktop\vlg-dataset\train"
test_folder = r"C:\Users\Lenovo\Desktop\vlg-dataset\test"

In [17]:
# Loading  a pre-trained ResNet-18 model
model = resnet18(weights=ResNet18_Weights.DEFAULT)

In [18]:
num_classes = 40 
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [19]:
#running the model on CUDA 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [21]:
#parameters
num_epochs = 20
batch_size = 128

In [23]:
#the changes made to the images 
changes_to_image = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(), #randomly flip some images 
    transforms.RandomRotation(10),      # Randomly rotate images by 10 degrees
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

In [24]:
#loading the data into the system from the files 

train_dataset = ImageFolder(root=train_folder, transform=changes_to_image)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#training loop of the model 

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

In [13]:
# predicting the images with the trained model 
model.eval()
test_predictions = []

for img_name in os.listdir(test_folder):
    img_path = os.path.join(test_folder, img_name)
    image = Image.open(img_path).convert('RGB')
    image = changes_to_image(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
        test_predictions.append((img_name, train_dataset.classes[predicted_class]))

In [14]:
#saving the predictions 
submission = pd.DataFrame(test_predictions, columns=["image_id", "class"])
submission.to_csv("submission2.csv", index=False)